<a href="https://colab.research.google.com/github/dbasrai/watchmyset/blob/master/WMS_train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!python -m pip install --user numpy scipy matplotlib ipython jupyter pandas sympy nose
!pip install python_speech_features
from tqdm import tqdm
from scipy.io import wavfile
from python_speech_features import mfcc, logfbank

import librosa
from keras.utils import to_categorical

#RNN

Z_mfcc = []
name = []
_min, _max = float('inf'), -float('inf')

df = pd.DataFrame(columns=['name', 'laugh', 'x'])
for filename in os.listdir('/content/gdrive/My Drive/Colab Notebooks/csv'):
  df = pd.concat([df, pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/csv/{0}'.format(filename))], ignore_index=True)

#df=df[df.x!=1]

for i in tqdm(range(df.shape[0])):
  signal, rate = librosa.load('/content/gdrive/My Drive/Colab Notebooks/dataset/{0}'.format(df.name.iloc[i]), sr=16000)
  mel = mfcc(signal[:rate], rate, numcep=13, nfilt=26, nfft=512)
  Z_mfcc.append(mel)
  _min = min(np.amin(Z_mfcc), _min)
  _max = max(np.amax(Z_mfcc), _max)

X = np.array(Z_mfcc)
X = (X - _min) / (_max - _min)
X = X.reshape(X.shape[0], X.shape[1], X.shape[2])

Y = np.array(df.laugh)
Y = to_categorical(Y, num_classes = 2)

np.save('/content/gdrive/My Drive/Colab Notebooks/np_arrays/x_large_all.npy', X)
np.save('/content/gdrive/My Drive/Colab Notebooks/np_arrays/y_large_all.npy', Y)

X.shape, Y.shape

In [0]:
X = np.load('/content/gdrive/My Drive/Colab Notebooks/np_arrays/x_large_all.npy')
Y = np.load('/content/gdrive/My Drive/Colab Notebooks/np_arrays/y_large_all.npy')

In [0]:
from keras.layers import Conv2D, MaxPool2D, Flatten, LSTM
from keras.layers import Dropout, Dense, TimeDistributed
from keras.models import Sequential
from sklearn.utils.class_weight import compute_class_weight
from keras.callbacks import ModelCheckpoint

#RNN

def get_recurrent_model():
  model = Sequential()
  model.add(LSTM(128, return_sequences=True, input_shape=input_shape))
  model.add(LSTM(128, return_sequences=True))
  model.add(Dropout(0.5))
  model.add(TimeDistributed(Dense(64, activation='relu')))
  model.add(TimeDistributed(Dense(32, activation='relu')))
  model.add(TimeDistributed(Dense(16, activation='relu')))
  model.add(TimeDistributed(Dense(8, activation='relu')))
  model.add(Flatten())
  model.add(Dense(2, activation='softmax'))
  model.summary()
  model.compile(loss='categorical_crossentropy', 
                optimizer='adam', 
                metrics=['acc'])
  return model

y_flat = np.argmax(Y, axis=1)
class_weight = compute_class_weight('balanced', np.unique(y_flat), y_flat)
input_shape = (X.shape[1], X.shape[2])
model = get_recurrent_model()

model_path = '/content/gdrive/My Drive/Colab Notebooks/models/WMS_model_6k.model'

checkpoint = ModelCheckpoint(model_path, monitor = 'acc', verbose = 1, 
                             mode = 'max', save_best_only=True, 
                             save_weights_only = False, period=1)

history = model.fit(X, Y, epochs=10, batch_size = 32, class_weight = class_weight, shuffle=True, callbacks=[checkpoint])

In [0]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Val_Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()